# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [3]:
# Use as many python cells as you wish to write your code

## Importações

In [4]:
import pandas as pd
import numpy as np

### Carregar os arquivos

In [5]:
health = pd.read_csv('user_health_data.csv')
supplements = pd.read_csv('supplement_usage.csv')
experiments = pd.read_csv('experiments.csv')
profiles = pd.read_csv('user_profiles.csv')

### Transformando os dados

In [6]:
health['sleep_hours'] = pd.to_numeric(
    health['sleep_hours'].astype(str).str.replace('h', '', case=False).str.strip(),
    errors='coerce'
)
health['date'] = pd.to_datetime(health['date'], errors='coerce')

supplements['dosage_grams'] = supplements['dosage'] / 1000
supplements['date'] = pd.to_datetime(supplements['date'], errors='coerce')
supplements = supplements.merge(
    experiments[['experiment_id', 'name']].rename(columns={'name': 'experiment_name'}),
    on='experiment_id', how='left'
)

df = health.merge(
    supplements[['user_id', 'date', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo']],
    on=['user_id', 'date'], how='outer'
)
df['supplement_name'] = df['supplement_name'].fillna('No intake')

### Classificação de idade

In [7]:
def classify_age(age):
    if pd.isna(age):
        return 'Unknown'
    age = int(age)
    if age < 18:
        return 'Under 18'
    elif age <= 25:
        return '18-25'
    elif age <= 35:
        return '26-35'
    elif age <= 45:
        return '36-45'
    elif age <= 55:
        return '46-55'
    elif age <= 65:
        return '56-65'
    else:
        return 'Over 65'

profiles['user_age_group'] = profiles['age'].apply(classify_age)

### merge com email e faixa etária

In [8]:
df = df.merge(profiles[['user_id', 'email', 'user_age_group']], on='user_id', how='left')

df = df.dropna(subset=['user_id', 'date'])

df = df[[
    'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
    'supplement_name', 'dosage_grams', 'is_placebo',
    'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'
]]

### Merge/df final

In [9]:
def merge_all_data(health_file, supplement_file, experiments_file, profiles_file):
    health = pd.read_csv(health_file)
    supplements = pd.read_csv(supplement_file)
    experiments = pd.read_csv(experiments_file)
    profiles = pd.read_csv(profiles_file)

    health['sleep_hours'] = pd.to_numeric(
        health['sleep_hours'].astype(str).str.replace('h', '', case=False).str.strip(),
        errors='coerce'
    )
    health['date'] = pd.to_datetime(health['date'], errors='coerce')

    supplements['dosage_grams'] = supplements['dosage'] / 1000
    supplements['date'] = pd.to_datetime(supplements['date'], errors='coerce')

    supplements = supplements.merge(
        experiments[['experiment_id', 'name']].rename(columns={'name': 'experiment_name'}),
        on='experiment_id', how='left'
    )

    df = health.merge(
        supplements[['user_id', 'date', 'experiment_name', 'supplement_name', 'dosage_grams', 'is_placebo']],
        on=['user_id', 'date'], how='outer'
    )

    df['supplement_name'] = df['supplement_name'].fillna('No intake')

    def classify_age(age):
        if pd.isna(age):
            return 'Unknown'
        age = int(age)
        if age < 18: return 'Under 18'
        elif age <= 25: return '18-25'
        elif age <= 35: return '26-35'
        elif age <= 45: return '36-45'
        elif age <= 55: return '46-55'
        elif age <= 65: return '56-65'
        else: return 'Over 65'

    profiles['user_age_group'] = profiles['age'].apply(classify_age)

    df = df.merge(profiles[['user_id', 'email', 'user_age_group']], on='user_id', how='left')
    df = df.dropna(subset=['user_id', 'date'])

    df = df[[
        'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
        'supplement_name', 'dosage_grams', 'is_placebo',
        'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'
    ]]

    return df

In [10]:
merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-01-31,user_65@myemail.com,26-35,Memory,Placebo,0.170989,False,84.172177,96.814856,11.4,2
1,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-02-28,user_65@myemail.com,26-35,Sleep Quality,Magnesium,0.494938,False,85.200747,130.836935,5.8,2
2,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-03-31,user_65@myemail.com,26-35,Sleep Quality,Placebo,0.184596,False,76.695318,109.782341,7.0,1
3,005dd64e-3863-49f3-93b0-1aea9a84d526,2018-04-30,user_65@myemail.com,26-35,Endurance,Omega-3,0.313256,True,65.772482,100.765545,7.5,1
4,016e4095-8b5d-43d9-83fd-e28b38be2f7d,2018-01-31,contact_390@email.com,36-45,NaN,No intake,NaN,NaN,86.762141,103.689177,8.2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2716,ff0e1eb5-1deb-4ddc-9ce9-548706be6bef,2018-04-30,contact_299@myemail.com,26-35,Endurance,Magnesium,0.163610,False,78.136649,86.144705,5.3,2
2717,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-01-31,user323@email.com,46-55,Strength,Vitamin C,0.407786,False,80.122904,134.196895,11.2,1
2718,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-02-28,user323@email.com,46-55,Recovery,Omega-3,0.350319,False,63.316583,79.378241,11.6,1
2719,ffaccfef-220a-4976-96f7-22e4f2fc3ecc,2018-03-31,user323@email.com,46-55,Recovery,Zinc,0.111641,False,84.800053,117.005827,11.1,3
